# Reventar a las casas de apuestas

### Imports and options

In [1]:
import sqlite3
import time
from bs4 import BeautifulSoup
from abc import ABC
import Levenshtein as lev
import string

In [2]:
# Source: https://github.com/JeffSackmann/tennis_atp y https://github.com/JeffSackmann/tennis_wta
"""
atp, wta, all = set(), set(), set()
for row in open("atp_players.csv"):
    player = row.split(',')
#    if player[4] == "" or int(player[4][:4]) <= 1980:
#        continue
    atp.add((player[1] + ' ' + player[2]).upper())
for row in open("wta_players.csv"):
    player = row.split(',')
#    if player[4] == "" or int(player[4][:4]) <= 1980:
#        continue
    wta.add((player[1] + ' ' + player[2]).upper())
all_players = atp.union(wta)
print(f"ATP total: {len(atp)}")
print(f"WTA total: {len(wta)}")
print(f"ALL total: {len(all_players)}")
"""

'\natp, wta, all = set(), set(), set()\nfor row in open("atp_players.csv"):\n    player = row.split(\',\')\n#    if player[4] == "" or int(player[4][:4]) <= 1980:\n#        continue\n    atp.add((player[1] + \' \' + player[2]).upper())\nfor row in open("wta_players.csv"):\n    player = row.split(\',\')\n#    if player[4] == "" or int(player[4][:4]) <= 1980:\n#        continue\n    wta.add((player[1] + \' \' + player[2]).upper())\nall_players = atp.union(wta)\nprint(f"ATP total: {len(atp)}")\nprint(f"WTA total: {len(wta)}")\nprint(f"ALL total: {len(all_players)}")\n'

In [3]:
conn = sqlite3.connect("bets_database.db")
cursor = conn.cursor()

#If the table does not exists, we create it
if cursor.execute("PRAGMA table_info(TENNIS_PLAYERS)").fetchall() == []:
    tennis_players_table_query = """CREATE TABLE TENNIS_PLAYERS (
        name TEXT     UNIQUE,
        sex  TEXT (1) 
    );"""
    cursor.execute(tennis_players_table_query)
    conn.commit()

In [95]:
all_players = set([x[0].upper() for x in cursor.execute("SELECT name FROM TENNIS_PLAYERS").fetchall()])

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC # Wait for condition

options = webdriver.ChromeOptions()
options.add_argument('ignore-certificate-errors')
options.add_argument("--window-size=1920,1080")
options.add_argument('--headless')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

#options.add_experimental_option('excludeSwitches', ['enable-logging'])

driver = webdriver.Chrome(options=options)

In [61]:
class Bet():
    def __init__(self, teams, bets):
        self.teams = [x.strip().upper() for x in teams]
        self.real_teams = []
        self.bets = [float(x.replace(',', '.')) for x in bets]
        self.date_time = None
        self.link = None
        self.house = None
    
    def is_error(self):
        if len(self.real_teams) < 2 or len(self.bets) < 2:
            return True
        return False

    def __repr__(self):
        if self.is_error():
            return "ERROR BET"
        return f"{self.real_teams[0]} ({self.bets[0]}) x {self.real_teams[1]} ({self.bets[1]})"

In [69]:
class House(ABC):
    def link_sport(self, sport):
        pass
    def sport_bets(self, sport):
        pass
    def real_names(self):
        pass

In [84]:
class Retabet(House):
    def __init__(self):
        self.link = "https://apuestas.retabet.es"
        self.sports = {"tennis" : "tenis-m8"}
        self.ret_bets = []

    def link_sport(self, sport):
        return f"{self.link}/deportes/{self.sports[sport]}"

    def sport_bets(self, sport):
        driver.get(self.link_sport(sport))
        response = driver.page_source
        soup = BeautifulSoup(response, "html.parser")
        table = soup.find('ul', attrs={'class':'module__events-cont jbgroup'}) #TODO findAll
        for row in table:
            row = row.find('a', attrs={'class':'jlink'})
            teams = row.find('div', attrs={'class':'module__event-info'})
            teams = teams.find('ul', attrs={'class':'module__event-players'})
            teams = list(map(lambda elem: elem.text, teams.findAll('li')))

            bets = row.find('div', attrs={'class':'list-events__element__bets'})
            bets = row.find('div', attrs={'class':'module__bets-list jbet'})
            bets = row.findAll('li', attrs={'class':'jo module__bets-li'})
            bets_list = []
            for bet, _ in zip(bets, range(2)):
                bet = bet.find('span', attrs={'class':'jpr module__bets-cuota'})
                bets_list.append(bet.text)

            bet = Bet(teams, bets_list)
            self.ret_bets.append(bet)
        self.real_names()
        return self.ret_bets
    
    def real_names(self):
        for bet in self.ret_bets:
            for team in bet.teams:
                cnt, l_f = 0, []
                for player in all_players:
                    if player[0] == team[0] and player.split()[-1] == team.split()[-1]:
                        cnt += 1
                        l_f.append((player, lev.jaro_winkler(bet.teams[0][::-1], player[::-1])))
                l_f.sort(key=lambda x: x[1], reverse=True)
                if cnt == 1:
                    bet.real_teams.append(l_f[0][0])
            print(bet)

In [85]:
%%time
reta = Retabet()
reta_tennis = reta.sport_bets("tennis")

MAX PURCELL (2.1) x YU-HSIOU HSU (1.65)
ROMAN SAFIULLIN (1.18) x PETER POLANSKY (4.3)
ERROR BET
ERROR BET
ERROR BET
JASON KUBLER (1.45) x JAY CLARKE (2.55)
ZARINA DIYAS (3.2) x CAMILA GIORGI (1.33)
YANNICK HANFMANN (1.8) x ARTHUR RINDERKNECH (1.95)
YULIA PUTINTSEVA (1.5) x KATERYNA KOZLOVA (2.5)
ERROR BET
GUIDO ANDREOZZI (1.72) x JELLE SELS (2.0)
STORM SANDERS (1.85) x GREET MINNEN (1.9)
FELIPE MELIGENI RODRIGUES ALVES (1.8) x DIMITAR KUZMANOV (1.9)
GONZALO LAMA (1.15) x SANDRO KOPP (4.1)
ERROR BET
ERROR BET
ANTOINE HOANG (1.45) x LUKÁŠ ROSOL (2.5)
ERROR BET
ERROR BET
NIKOLOZ BASILASHVILI (2.35) x LASLO DJERE (1.55)
ERROR BET
ERROR BET
CASPER RUUD (1.14) x HENRI LAAKSONEN (5.0)
PANNA UDVARDY (3.2) x ANHELINA KALININA (1.3)
ERROR BET
ERROR BET
NICOLAS JARRY (1.4) x MATHEUS PUCINELLI DE ALMEIDA (2.8)
JESPER DE JONG (2.85) x BOTIC VAN DE ZANDSCHULP (1.37)
ROBERTO CARBALLÉS BAENA (1.65) x NORBERT GOMBOS (2.1)
PAULA ORMAECHEA (4.7) x DANIELLE COLLINS (1.15)
ERROR BET
BENOÎT PAIRE (3.6) x FE

In [74]:
%%time
reta.ret_bets

Wall time: 0 ns


[MAX PURCELL (2.1) x MAX PURCELL (1.65),
 ROMAN SAFIULLIN (1.18) x ROMAN SAFIULLIN (4.3),
 BORNA GOJO (1.95) x BORNA GOJO (1.75),
 ERROR BET,
 ERROR BET,
 JASON KUBLER (1.45) x JASON KUBLER (2.55),
 ZARINA DIYAS (3.2) x ZARINA DIYAS (1.33),
 YANNICK HANFMANN (1.8) x YANNICK HANFMANN (1.95),
 YULIA PUTINTSEVA (1.5) x YULIA PUTINTSEVA (2.5),
 ERROR BET,
 GUIDO ANDREOZZI (1.75) x GUIDO ANDREOZZI (2.0),
 STORM SANDERS (1.85) x STORM SANDERS (1.9),
 FELIPE MELIGENI RODRIGUES ALVES (1.8) x FELIPE MELIGENI RODRIGUES ALVES (1.9),
 GONZALO LAMA (1.15) x GONZALO LAMA (4.1),
 ERROR BET,
 KIRILL KIVATTSEV (1.15) x KIRILL KIVATTSEV (4.2),
 ANTOINE HOANG (1.45) x ANTOINE HOANG (2.5),
 FEDERICO CORIA (3.3) x FEDERICO CORIA (1.3),
 ERROR BET,
 NIKOLOZ BASILASHVILI (2.35) x NIKOLOZ BASILASHVILI (1.55),
 ERROR BET,
 GRACE MIN (2.85) x GRACE MIN (1.4),
 CASPER RUUD (1.14) x CASPER RUUD (5.0),
 PANNA UDVARDY (3.2) x PANNA UDVARDY (1.3),
 ERROR BET,
 STEFANOS TSITSIPAS (1.12) x STEFANOS TSITSIPAS (5.5),
 N

In [86]:
class Bwin(House):
    def __init__(self):
        self.link = "https://sports.bwin.es/es/sports"
        self.sports = {"tennis" : "tenis-5"}

    def link_sport(self, sport):
        return f"{self.link}/{self.sports[sport]}"

    def sport_bets(self, sport):
        driver.get(self.link_sport(sport))
        while True:
            try:
                driver.find_element_by_css_selector('#main-view > ms-widget-layout > ms-widget-slot.col-9.widget-slot')
                break
            except:
                time.sleep(0.5)
        response = driver.page_source
        soup = BeautifulSoup(response, "html.parser")
        table1 = soup.find('ms-widget-slot', attrs={'class':'col-9 widget-slot'})
        table2 = table1.find('ms-tabbed-grid', attrs={'class':'grid-wrapper tabbed-grid-widget'})
        table3 = table2.find('ms-grid')
        table4 = table3.find('ms-event-group', attrs={'class':'event-group'})
        rows = table4.findAll('ms-event', attrs={'class':'grid-event ms-active-highlight'})
        
        ret_bets = []
        for row in rows:
            try:
                row = row.find('div', attrs={'class':'grid-event-wrapper'})
                
                teams = row.find('a', attrs={'class':'grid-info-wrapper fixed'})
                teams = teams.find('ms-event-detail', attrs={'class':'grid-event-detail'})
                teams = teams.find('ms-event-name', attrs={'class':'grid-event-name'})
                teams = teams.find('ms-inline-tooltip')
                teams = teams.find('div', attrs={'class':'participants-pair-game'})
                teams = teams.findAll('div', {'class':['participant-wrapper ng-star-inserted', 'participant-wrapper']})
                teams_final = []
                for team in teams:
                    team = team.find('div', attrs={'class':'participant'})
                    team = team.find(text=True, recursive=False).strip()
                    teams_final.append(team)

                bets = row.find('div', attrs={'class':'grid-group-container'})
                bets = row.find('ms-option-group', attrs={'class':'grid-option-group grid-group'})
                bets = row.findAll('ms-option', attrs={'class':'grid-option'})
                bets_final = []
                for bet, _ in zip(bets, range(2)):
                    bets_final.append(bet.text)

                bet = Bet(teams_final, bets_final)
                ret_bets.append(bet)
            except:
                pass
        return ret_bets

In [87]:
%%time
bwin = Bwin()
bwin_tennis = bwin.sport_bets("tennis")

Wall time: 4.46 s


In [88]:
bwin_tennis

[ERROR BET,
 ERROR BET,
 ERROR BET,
 ERROR BET,
 ERROR BET,
 ERROR BET,
 ERROR BET,
 ERROR BET,
 ERROR BET,
 ERROR BET,
 ERROR BET,
 ERROR BET,
 ERROR BET,
 ERROR BET,
 ERROR BET]

In [94]:
%%time
for bet in bwin_tennis:
    print(bet.teams[0])
    if bet.teams[0] in all_players:
        print("YES")
    else:
        print("NO")

KAMILLA RAKHIMOVA
NO
CAROLINE GARCIA
NO
MARYNA ZANEVSKA
NO
ALYCIA PARKS
NO
ZARINA DIYAS
NO
YANNICK HANFMANN
NO
NIKOLOZ BASILASHVILI
NO
FEDERICO CORIA
NO
STEFANOS TSITSIPAS
NO
CASPER RUUD
NO
DUSAN LAJOVIC
NO
ROBERTO CARBALLES BAENA
NO
BENOIT PAIRE
NO
JENSON BROOKSBY
NO
JORDAN THOMPSON
NO
Wall time: 1.15 ms


In [16]:
class WilliamHill(House):
    def __init__(self):
        self.link = "https://sports.williamhill.es/betting/es-es"
        self.sports = {"tennis" : "tenis/partidos"}

    def link_sport(self, sport):
        return f"{self.link}/{self.sports[sport]}"

    def sport_bets(self, sport):
        driver.get(self.link_sport(sport))
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.2)
        response = driver.page_source
        soup = BeautifulSoup(response, "html.parser")
        matches = soup.findAll('div', attrs={'class':'event'})
        ret_bets = []
        for match in matches:
            teams = match.find('a')
            teams_final, bets_final = [], []
            teams_final = teams['title'].split('₋')
            bets = match.findAll('button', attrs={'class':'btn betbutton oddsbutton'})
            for bet in bets:
                bets_final.append(bet.text)
            ret_bets.append(Bet(teams_final, bets_final))
        return ret_bets

In [17]:
%%time
william_hill = WilliamHill()
william_hill_tennis = william_hill.sport_bets("tennis")

Wall time: 4.17 s


In [18]:
william_hill_tennis

[ALEXANDRA PANOVA/JULIA WACHACZYK (1.5) x FRANCESCA DI LORENZO/ASTRA SHARMA (2.5),
 MAX PURCELL (2.2) x YU HSIOU HSU (1.61),
 ROMAN SAFIULLIN (1.14) x PETER POLANSKY (5.0),
 EKATERINA SHALIMOVA (4.0) x ANASTASIA TIKHONOVA (1.22),
 MARIAM BOLKVADZE (1.33) x SHALIMAR TALBI (3.25),
 YULIYA HATOUKA (1.73) x ANNA KUBAREVA (2.0),
 MATTHEW SEGURA (1.53) x ALEX CAIRO (2.37),
 LUKAS GREIF (5.5) x AIDAN KIM (1.12)]

In [19]:
driver.get("https://m.apuestas.codere.es/deportes/index.htm#/HomePage")

In [20]:
driver.get("https://betway.es/es/sports")